In [1]:
import os
import glob
import numpy as np
import nibabel as nib
import SimpleITK as sitk
from nilearn import image
from scipy.ndimage import affine_transform
from scipy.spatial.distance import directed_hausdorff

In [2]:
# Specify input and output path
# For LINUX:
path ='ai4mi_project/data/segthor_train/train/'

# For WSL:
# path = '/home/{user}/ai4mi_project/data/segthor_train/train/'

In [17]:
def create_augmentation(folder_path, patients_tobe_augmented, new_patients_name, degree=10, aug='affine', fwhm=4, use_default_matrix=True, give_matrix=None, threshold=-500):
    """
    Example of how the variables should look:
    folder_path (string): 'ai4mi_project/data/segthor_train/train/'
    patients_tobe_augmented (list of strings): ['03', '11, '27']
    new_patients_name (list of strings): ['41', '42', '43']
    degree can be (int): 10     
    aug (string): "affine" or "gaussian" or "threshold"
    
    fwhm (int): 4 
    # NiLearn has its own function for applying Gaussian spatial smoothing to images as well. 
    # The only real difference from scipy.ndimage’s gaussian_filter() function is that instead of 
    # specifying the smoothing kernel in standard deviations, we specify it in units of 
    # full width half-maximum (FWHM). This is the standard way that most neuroimaging analysis packages 
    # specify smoothing kernel size, so it is preferable to SciPy’s approach. As the term implies, 
    # FWHM is the width of the smoothing kernel, in millimetres, at the point in the kernel where it is half 
    # of its maximum height. Thus a larger FWHM value applies more smoothing.
    use_default_matrix can be set to False and a different matrix can be given to parameter give_matrix
    
    threshold (int between -1000 and 31743): -500 (Everything belowe becomes zero
    """
    if use_default_matrix:
        theta = np.radians(degree)  # 10 degree rotation
        matrix = np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                           [np.sin(theta), np.cos(theta),  0, 0],
                           [0,             0,              1, 0],
                           [0,             0,              0, 1]])
    else:
        matrix = give_matrix
           
    for patient, new_name in zip(patients_tobe_augmented, new_patients_name):
        
        ct = nib.load(folder_path + 'Patient_' + patient + '/Patient_' + patient + '.nii.gz')
        gt = nib.load(folder_path + 'Patient_' + patient + '/GT.nii.gz')

        # Creating directory for new patient
        os.makedirs(folder_path + 'Patient_' + new_name, exist_ok=True)
        
        if aug == 'affine':
            ct_new_img = image.resample_img(ct, target_affine=matrix @ ct.affine, target_shape=ct.shape)
            gt_new_img = image.resample_img(gt, target_affine=matrix @ gt.affine, target_shape=gt.shape)

        elif aug == 'gaussian':
            ct_new_img = image.smooth_img(ct, fwhm)
            gt_new_img = gt

        elif aug == 'threshold':
            data = ct.get_fdata() 
            data[data < threshold] = 0
            ct_new_img = nib.Nifti1Image(data, ct.affine)
            gt_new_img = gt

        # Save the transformed image
        nib.save(ct_new_img, folder_path + 'Patient_' + new_name + '/Patient_' + new_name + '.nii.gz')
        nib.save(gt_new_img, folder_path + 'Patient_' + new_name + '/GT.nii.gz')


patients_tobe = ['27']
new_patients = ['41']
create_augmentation(path, patients_tobe, new_patients, aug='threshold')